In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


c:\Users\shahn\anaconda3\envs\major_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

True
0
NVIDIA GeForce MX450


In [3]:
import pinecone
from langchain.vectorstores import Pinecone


In [5]:
db = Pinecone.from_existing_index("medical-data", embedding_model)
ans=db.similarity_search("What are the common symptoms of diabetes?",k=4)

In [6]:
print(ans[0].page_content)

• Type I diabetes mellitus. Characterized by fatigue and
an abnormally high level of glucose in the blood(hyperglycemia).
• Amyotrophic lateral schlerosis. First signs are stum-
bling and difficulty climbing stairs. Later, musclecramps and twitching may be observed as well asweakness in the hands making fastening buttons orturning a key difficult. Speech may become slowed orslurred. There may also be difficluty swallowing. Asrespiratory muscles atrophy, there is increased dangerof aspiration or lung infection.
• Guillain-Barre syndrome. Muscle weakness in the legs
occurs first, then the arms and face. Paresthesias (aprickly, tingling sensation) is also felt. This disorderaffects both sides of the body and may involve paralysisand the muscles that control breathing.


In [7]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub

In [8]:
template = """
You help everyone by answering questions, and improve your answers from previous answers in History.
Don't try to make up an answer, if you don't know, just say that you don't know.
Answer in the same language the question was asked.
Answer in a way that is easy to understand.
Do not say "Based on the information you provided, ..." or "I think the answer is...".
Just answer the question directly in detail.

History: {chat_history}

Context: {context}

Question: {query}
Answer: 
"""

In [9]:
from langchain.llms import CTransformers

config = {'max_new_tokens': 512,'context_length': 8000}

llm = CTransformers( 
                    model='TheBloke/Llama-2-7B-Chat-GGUF',
                    model_file="llama-2-7b-chat.Q4_K_S.gguf",
                    config=config,
                    model_type='llama',
                    device = 'cuda',
                    temprature=0.8 )


Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [10]:
from accelerate import Accelerator
accelerator = Accelerator()
llm, config = accelerator.prepare(llm, config)

In [15]:

memory=ConversationBufferMemory(memory_key='chat_history',return_messages=True)
retriever = db.as_retriever(search_kwargs={"k":3})
conversation=ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        chain_type="stuff",
        return_source_documents=True,
        get_chat_history=lambda h : h,
        combine_docs_chain_kwargs={'prompt': PROMPT}
    )

NameError: name 'PROMPT' is not defined

In [16]:
chat_history=memory.load_memory_variables({})
chat_history

{'chat_history': []}

In [17]:
query="What are the typical symptoms of a skin infection?"

In [18]:
db = Pinecone.from_existing_index("medical-data", embedding_model)
ans=db.similarity_search(query,k=2)
context=""
for i in ans:
    context+=f"Document: {i.page_content} \n \n"
context

'Document: contracted, but the symptoms usually appear within oneweek of exposure.\nCutaneous anthrax\nIn humans, anthrax usually occurs when the bacteria\nenter a cut or abrasion, causing a skin (cutaneous) infec-tion at the site. Cutaneous anthrax, as this infection iscalled, is the mildest form of the disease. At first, thebacteria cause an itchy, raised area like an insect bite.Within one to two days, inflammation occurs around theraised area, and a blister forms around an area of dyingtissue that becomes black in the center. Other symptomsmay include shivering and chills. In most cases the bacte-ria remain within the sore. If, however, they spread to thenearest lymph node (or, in rare cases, escape into thebloodstream), the bacteria can cause a form of blood poi-\nsoning that rapidly proves fatal. \n \nDocument: heart, spinal cord, and lungs, and in the 1990s has beencalled the “flesh-eating bacterium.” Staphylococci bacte-ria may also cause lymphangitis.\nAlthough anyone can deve

In [19]:

response=convo({"question":query,'context': context,'chat_history':chat_history})
print(response['answer'])

NameError: name 'convo' is not defined

In [ ]:
# 2-nd approach

In [11]:
template = """
You are a doctor who help everyone by answering their questions related to their ailment, and improve your answers from previous answers in History.
Don't try to make up an answer, if you don't know, just say that you don't know.
Answer in a way how doctor answers to their patient. 
Do not say  " based on the information provided in the passage", Based on the information you provided, ..." or "I think the answer is...". Just answer the question directly in detail.

History: {chat_history}

Context: {context}

Question: {query}
Answer: 
"""

In [12]:
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(input_variables=["chat_history", "query", "context"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="query")

chain = load_qa_chain(llm, chain_type="stuff", memory=memory, prompt=prompt)

In [13]:
query="Can you suggest some exercise for relieveing chest pain"

In [14]:
chat_history=memory.load_memory_variables({})['chat_history']

chat_history

''

In [15]:
chats=chat_history.split('Human:')[1:]
human_chats=[]
AI_chats=[]
for chat in chats:
    a=chat.split('AI:')
    human_chats.append(a[0])
    AI_chats.append(a[1])

human_chats

[]

In [16]:
temp_chat=chat_history.replace("Human:","")
temp_chat=temp_chat.replace("AI:","")
temp_chat

''

In [17]:

search=temp_chat + " \n" + query
db = Pinecone.from_existing_index("medical-data", embedding_model)
ans=db.similarity_search(search,k=3)
context=""
for i in ans:
    context+=f"Document: {i.page_content} \n \n"
context

'Document: Treatment\nTreatment should involve efforts to resolve any\nunderlying disorder. Infections will require antibiotics ,\nobstruction may require the removal of a foreign object ortumor. Medications are available to help thin the sputum,so that it can be more effectively coughed up. Rhythmicclapping on the chest and back, while the patient assumesa number of positions (head down, primarily), may helpthe lungs to drain more effectively. This is called chest\nphysical therapy , or percussion and postural drainage.\nWhen a particular area of the lung is constantly and \n \nDocument: rest or choose exercises that put less stress on their feet, at\nleast until the misalignment is corrected. In some cases,physicians also use steroid injections with local anestheticaround the bunion to reduce inflammation.\nSurgery\nIf conservative treatment is not successful, surgical \n \nDocument: build strength and health, and also advises massage thera-py and self-massage as ways of increasing c

In [18]:


response = chain({"input_documents": ans,"query":query,"context": context,'chat_history':chat_history})

print("Model's Response:", response)

c:\Users\shahn\anaconda3\envs\major_project\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [72]:
print(response['output_text'])

Thank you for sharing your concerns with me. Based on the information provided, it's difficult to pinpoint the exact cause of your symptoms without further evaluation and diagnostic testing. However, I can provide some potential explanations and recommendations based on your symptoms.
Firstly, chest pain or discomfort can be caused by various factors, including cardiac arrhythmias, coronary artery disease, pulmonary conditions, and musculoskeletal issues. Since you have experienced pain in the left side of your chest, it's possible that it could be related to a cardiac issue such as atrial fibrillation or angina.
Angina is a common condition characterized by chest discomfort or pain that occurs when the heart muscle does not receive enough oxygen-rich blood. This can happen due to blockages in the coronary arteries, which supply blood to the heart. The pain can radiate to other areas such as the arm, neck, jaw, or back. Based on your symptoms of mild chest pain and sensation in the upp

In [28]:
print(response)

{'input_documents': [Document(page_content='begin to fall. A person with diabetes mellitus either does\nnot make enough insulin, or makes insulin that does notwork properly. The result is blood sugar that remainshigh, a condition called hyperglycemia.\nDiabetes must be diagnosed as early as possible. If\nleft untreated, it can damage or cause failure of the eyes,kidneys, nerves, heart, blood vessels, and other bodyorgans. Hypoglycemia , or low blood sugar, may also be\ndiscovered through blood sugar testing. Hypoglycemia iscaused by various hormone disorders and liver disease,as well as by too much insulin.\nDescription\nThere are a variety of ways to measure a person’s\nblood sugar.\nWhole blood glucose test\nWhole blood glucose testing can be performed by a'), Document(page_content='Nancy Ross-Flanigan\nAntidiabetic drugs\nDefinition\nAntidiabetic drugs are medicines that help control\nblood sugar levels in people with diabetes mellitus\n(sugar diabetes).\nPurpose\nDiabetes may be di